In [1]:
import evaluate
import tokenizers
import numpy as np
import pandas as pd
import transformers
from datetime import datetime
from datasets import load_dataset, load_from_disk, concatenate_datasets
from transformers import AutoTokenizer, DataCollatorForLanguageModeling, AlbertForMaskedLM, AutoConfig, TrainingArguments, Trainer, pipeline

Carregando dataset:

In [2]:
dataset = load_from_disk('../CLIdata/datasets/cuneiform-spaced-indexed')

Carregando o tokenizer:

In [3]:
my_tokenizer = AutoTokenizer.from_pretrained('../tokenizers/bert-base-uncased_train_val_test_maxlen_512_vocab_size_1000')
vocab_size = my_tokenizer.vocab_size
max_len = my_tokenizer.model_max_length

Criamos uma função que identifica os tokens dos textos com índices:

In [4]:
def tokenize_function(example):
    output = my_tokenizer(example["text"], truncation=True, max_length=max_len)
    input_batch = []
    for token_id in output['input_ids']:
        input_batch.append(token_id)
    return {"input_ids": input_batch}

In [5]:
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset["train"].column_names)

Loading cached processed dataset at ../CLIdata/datasets/cuneiform-spaced-indexed/train\cache-585e058a74ca0651.arrow
Loading cached processed dataset at ../CLIdata/datasets/cuneiform-spaced-indexed/val\cache-95b8b34ca1812b1e.arrow


  0%|          | 0/7 [00:00<?, ?ba/s]

In [6]:
data_collator = DataCollatorForLanguageModeling(tokenizer=my_tokenizer, mlm=True, mlm_probability=0.15)

In [7]:
config = AutoConfig.from_pretrained('albert-base-v2')
model = AlbertForMaskedLM(config)

In [32]:
dia = datetime.today().strftime("%Y-%M-%d")
hora = datetime.now().strftime("%H-%M")

batch = 16
training_args = TrainingArguments(
    output_dir=f'../checkpoints-albert/standard_batch{batch}_{dia}_hora_{hora}',
    overwrite_output_dir=True,
    per_device_train_batch_size=batch,
    learning_rate=1e-5,
    logging_strategy='steps',
    logging_steps=1e3,
    save_strategy='steps',
    save_steps=10_000, 
    max_steps= 100_000
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=concatenate_datasets([tokenized_dataset['train'],tokenized_dataset['val'],tokenized_dataset['test']])
)

max_steps is given, it will override any value given in num_train_epochs


In [33]:
trainer.train()

c:\Users\igorr\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 98025
  Num Epochs = 17
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 100000
  Number of trainable parameters = 11221680


  0%|          | 0/100000 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 9.1386, 'learning_rate': 9.9e-06, 'epoch': 0.16}
{'loss': 7.2398, 'learning_rate': 9.800000000000001e-06, 'epoch': 0.33}
{'loss': 5.884, 'learning_rate': 9.7e-06, 'epoch': 0.49}
{'loss': 5.2498, 'learning_rate': 9.600000000000001e-06, 'epoch': 0.65}
{'loss': 4.9791, 'learning_rate': 9.5e-06, 'epoch': 0.82}
{'loss': 4.8579, 'learning_rate': 9.4e-06, 'epoch': 0.98}
{'loss': 4.7726, 'learning_rate': 9.3e-06, 'epoch': 1.14}
{'loss': 4.7219, 'learning_rate': 9.200000000000002e-06, 'epoch': 1.31}
{'loss': 4.671, 'learning_rate': 9.100000000000001e-06, 'epoch': 1.47}


Saving model checkpoint to ../checkpoints-albert/standard_2022-45-13_hora_22-45\checkpoint-10000
Configuration saved in ../checkpoints-albert/standard_2022-45-13_hora_22-45\checkpoint-10000\config.json
Model weights saved in ../checkpoints-albert/standard_2022-45-13_hora_22-45\checkpoint-10000\pytorch_model.bin


...

Saving model checkpoint to ../checkpoints-albert/standard_2022-45-13_hora_22-45\checkpoint-90000
Configuration saved in ../checkpoints-albert/standard_2022-45-13_hora_22-45\checkpoint-90000\config.json
Model weights saved in ../checkpoints-albert/standard_2022-45-13_hora_22-45\checkpoint-90000\pytorch_model.bin


{'loss': 3.5963, 'learning_rate': 1.0000000000000002e-06, 'epoch': 14.69}
{'loss': 3.5634, 'learning_rate': 9.000000000000001e-07, 'epoch': 14.85}
{'loss': 3.557, 'learning_rate': 8.000000000000001e-07, 'epoch': 15.02}
{'loss': 3.5716, 'learning_rate': 7.000000000000001e-07, 'epoch': 15.18}
{'loss': 3.5871, 'learning_rate': 6.000000000000001e-07, 'epoch': 15.34}
{'loss': 3.5734, 'learning_rate': 5.000000000000001e-07, 'epoch': 15.51}
{'loss': 3.5658, 'learning_rate': 4.0000000000000003e-07, 'epoch': 15.67}
{'loss': 3.5825, 'learning_rate': 3.0000000000000004e-07, 'epoch': 15.83}
{'loss': 3.5668, 'learning_rate': 2.0000000000000002e-07, 'epoch': 15.99}
{'loss': 3.5574, 'learning_rate': 1.0000000000000001e-07, 'epoch': 16.16}


Saving model checkpoint to ../checkpoints-albert/standard_2022-45-13_hora_22-45\checkpoint-100000
Configuration saved in ../checkpoints-albert/standard_2022-45-13_hora_22-45\checkpoint-100000\config.json
Model weights saved in ../checkpoints-albert/standard_2022-45-13_hora_22-45\checkpoint-100000\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




{'loss': 3.5532, 'learning_rate': 0.0, 'epoch': 16.32}
{'train_runtime': 9804.3097, 'train_samples_per_second': 163.194, 'train_steps_per_second': 10.2, 'train_loss': 4.048913425292969, 'epoch': 16.32}


TrainOutput(global_step=100000, training_loss=4.048913425292969, metrics={'train_runtime': 9804.3097, 'train_samples_per_second': 163.194, 'train_steps_per_second': 10.2, 'train_loss': 4.048913425292969, 'epoch': 16.32})

In [8]:
config = AutoConfig.from_pretrained('../checkpoints-albert/standard_2022-45-13_hora_22-45/checkpoint-100000/')
model2 = AlbertForMaskedLM(config)

In [9]:
dia = datetime.today().strftime("%Y-%M-%d")
hora = datetime.now().strftime("%H-%M")

batch = 16
lr = 1e-5
training_args = TrainingArguments(
    output_dir=f'../checkpoints-albert/standard_2022-45-13_hora_22-45',
    overwrite_output_dir=True,
    per_device_train_batch_size=batch,
    learning_rate=lr,
    logging_strategy='steps',
    logging_steps=1e3,
    save_strategy='steps',
    save_steps=10_000, 
    max_steps= 1_000_000
)

trainer = Trainer(
    model=model2,
    args=training_args,
    data_collator=data_collator,
    train_dataset=concatenate_datasets([tokenized_dataset['train'],tokenized_dataset['val'],tokenized_dataset['test']])
)

max_steps is given, it will override any value given in num_train_epochs


In [10]:
trainer.train(resume_from_checkpoint = True)

Loading model from ../checkpoints-albert/standard_2022-45-13_hora_22-45\checkpoint-100000.
c:\Users\igorr\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 98025
  Num Epochs = 164
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1000000
  Number of trainable parameters = 11221680
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 16
  Continuing training from global step 100000
  Will skip the first 16 epochs then the first 1968 batches in the first epoch. If this takes a lot of time, you c

  0%|          | 0/1968 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 3.6283, 'learning_rate': 8.99e-06, 'epoch': 16.48}
{'loss': 3.6444, 'learning_rate': 8.98e-06, 'epoch': 16.65}
{'loss': 3.6182, 'learning_rate': 8.97e-06, 'epoch': 16.81}
{'loss': 3.625, 'learning_rate': 8.96e-06, 'epoch': 16.97}
{'loss': 3.5537, 'learning_rate': 8.95e-06, 'epoch': 17.14}
{'loss': 3.582, 'learning_rate': 8.94e-06, 'epoch': 17.3}
{'loss': 3.5884, 'learning_rate': 8.930000000000001e-06, 'epoch': 17.46}
{'loss': 3.5586, 'learning_rate': 8.920000000000001e-06, 'epoch': 17.63}
{'loss': 3.5367, 'learning_rate': 8.910000000000001e-06, 'epoch': 17.79}


...

Saving model checkpoint to ../checkpoints-albert/standard_2022-45-13_hora_22-45\checkpoint-850000
Configuration saved in ../checkpoints-albert/standard_2022-45-13_hora_22-45\checkpoint-850000\config.json
Model weights saved in ../checkpoints-albert/standard_2022-45-13_hora_22-45\checkpoint-850000\pytorch_model.bin


{'loss': 1.9037, 'learning_rate': 1.5e-06, 'epoch': 138.73}
{'loss': 1.9223, 'learning_rate': 1.4900000000000001e-06, 'epoch': 138.89}
{'loss': 1.9047, 'learning_rate': 1.48e-06, 'epoch': 139.06}
{'loss': 1.8995, 'learning_rate': 1.4700000000000001e-06, 'epoch': 139.22}
{'loss': 1.9247, 'learning_rate': 1.46e-06, 'epoch': 139.38}
{'loss': 1.9229, 'learning_rate': 1.45e-06, 'epoch': 139.55}
{'loss': 1.9422, 'learning_rate': 1.44e-06, 'epoch': 139.71}
{'loss': 1.9114, 'learning_rate': 1.43e-06, 'epoch': 139.87}
{'loss': 1.9485, 'learning_rate': 1.42e-06, 'epoch': 140.04}
{'loss': 1.9096, 'learning_rate': 1.41e-06, 'epoch': 140.2}


Saving model checkpoint to ../checkpoints-albert/standard_2022-45-13_hora_22-45\checkpoint-860000
Configuration saved in ../checkpoints-albert/standard_2022-45-13_hora_22-45\checkpoint-860000\config.json
Model weights saved in ../checkpoints-albert/standard_2022-45-13_hora_22-45\checkpoint-860000\pytorch_model.bin


{'loss': 1.9232, 'learning_rate': 1.4000000000000001e-06, 'epoch': 140.36}
{'loss': 1.908, 'learning_rate': 1.3900000000000002e-06, 'epoch': 140.53}
{'loss': 1.8975, 'learning_rate': 1.3800000000000001e-06, 'epoch': 140.69}
{'loss': 1.9163, 'learning_rate': 1.3700000000000002e-06, 'epoch': 140.85}
{'loss': 1.908, 'learning_rate': 1.3600000000000001e-06, 'epoch': 141.02}


KeyboardInterrupt: 